In [ ]:
import sys
import os

# required file paths
portfolio_dir = "../../data/portfolio"
config_file = os.path.join(portfolio_dir, "config.yml")
holdings_files = os.path.join(portfolio_dir, "holdings")
factor_weights_file = os.path.join(portfolio_dir, "asset_class_weights_matrix.csv")

print("portfolio_dir:", portfolio_dir)
print("config_file:", config_file)
print("factor_weights_file:", factor_weights_file)
print("holdings_files:", holdings_files)

import yfinance
print("yfinance version:", yfinance.__version__)

In [ ]:
from portopt.config import load_config

# load config
config = load_config(config_file)
import pprint
pprint.pprint(config)

In [ ]:
from portopt.portfolio import Portfolio

# create portfolio object
portfolio = Portfolio(config, factor_weights_file, holdings_files)
portfolio

In [ ]:
# load tables
verbose = True

# load dimensions
accounts = portfolio.getAccounts().reset_index()
factors = portfolio.getFactors().reset_index()
tickers = portfolio.getTickers(verbose=verbose).reset_index()

# load facts
holdings = portfolio.getHoldings(verbose=verbose).reset_index()
prices = portfolio.getPrices(verbose=verbose).reset_index()

# load factor weights
factor_weights = portfolio.getFactorWeights().reset_index()


In [5]:
# register tables
import ibis

# Create Ibis DuckDB connection
con = ibis.duckdb.connect()

# Register DataFrames as tables
tables = {}
tables["accounts"] = con.create_table("accounts", accounts)
tables["factors"] = con.create_table("factors", factors)
tables["tickers"] = con.create_table("tickers", tickers)
tables["holdings"] = con.create_table("holdings", holdings)
tables["prices"] = con.create_table("prices", prices)
tables["factor_weights"] = con.create_table("factor_weights", factor_weights)

In [ ]:
from typing import List, Dict, Optional, Union

def _build_base_query(tables: Dict[str, ibis.Table],
                      dims: List[str],
                      filters: Dict[str, Union[str, List[str]]],
                      verbose = False) -> ibis.Table:
    # deteremine whether factor and factor_weights are required
    requires_factor_weights = False
    requires_factor_levels = False
    if dims:
        requires_factor_weights = any(d.startswith('Level_') or d == 'Factor' for d in dims)
        requires_factor_levels = any(d.startswith('Level_') for d in dims)
    if filters:
        requires_factor_weights = requires_factor_weights or any(d.startswith('Level_') or d == 'Factor' for d in filters.keys())
        requires_factor_levels = requires_factor_levels or any(d.startswith('Level_') for d in filters.keys())

    # base query
    query = tables["holdings"].join(tables["prices"], "Ticker")
    if verbose:
        print("Base Query --------------------------------")
        print(ibis.to_sql(query))

    # Joins
    # add factor tables if required
    if requires_factor_weights:
        # factor weights required for Factor and Level_* dimensions
        # - factor weights allows Level_* dimensions to be joined with holdings via Ticker
        query = query.join(tables["factor_weights"], "Ticker")
        if requires_factor_levels:
            query = query.join(tables["factors"], "Factor")

    if verbose:
        print("Joins Added--------------------------------")
        print(ibis.to_sql(query))

    return query

def _apply_filters(
    query: ibis.Table,
    filters: Optional[Dict[str, Union[str, List[str]]]] = None,
    verbose = False
) -> ibis.Table:
    """Apply dimension filters to the query.
    
    Args:
        query: Query to filter
        filters: Dict of filters to apply
            
    Returns:
        Filtered query
    """
    if not filters:
        return query

    for dim, values in filters.items():
        # Convert single values to list
        if isinstance(values, str):
            values = [values]
                
        # Apply filter
        query = query.filter(getattr(query, dim).isin(values))

    if verbose:
        print("Filters Added--------------------------------")
        print(ibis.to_sql(query))

    return query

def _build_total_value_subquery(unfiltered_query: ibis.Table,
                       filtered_query: ibis.Table,
                       portfolio_allocation: bool,
                       agg_by_factors: bool,
                       verbose: bool = False) -> ibis.Table:
    """Calculate total value used to calculate Allocation metric.
    
    Args:
        unfiltered_query: Query to calculate total value for
        filtered_query: Query to calculate total value for
        portfolio_allocation: Whether to calculate total value for portfolio allocation
        agg_by_factors: Whether to aggregate by factors
    """
    # identify correct total_value query - unfiltered or filtered
    if portfolio_allocation:
        # use UNFILTERED query to calculate total value used to calculate Allocation
        total_query = unfiltered_query
    else:
        # use FILTERED query to calculate total value used to calculate Allocation
        total_query = filtered_query

    # build total value expression - used to calculate Allocation
    if agg_by_factors:
        total_value = (total_query.Quantity * total_query.Price * total_query.Weight).sum()
    else:
        total_value = (total_query.Quantity * total_query.Price).sum()

    if verbose:
        print("Total Value Query Created--------------------------------")
        print(ibis.to_sql(total_value))

    return total_value.name("TotalValue")  # Not executed yet, will be used in expression

def _build_group_by(query: ibis.Table,
                    dims: List[str],
                    metrics: List[str],
                    agg_by_factors: bool,
                    verbose: bool = False) -> ibis.Table:
    """Build aggregate expressions for the query.
    
    Args:
        query: Query to build aggregates for
        metrics: List of metrics to build aggregates for
        agg_by_factors: Whether to aggregate by factors
    """
    agg_exprs = []
    if 'Quantity' in metrics:
        agg_exprs.append(query.Quantity.sum().name("Quantity"))

    if 'Value' in metrics or 'Allocation' in metrics:
        if agg_by_factors:
            agg_exprs.append((query.Quantity * query.Price * query.Weight).sum().name("Value"))
        else:   
            agg_exprs.append((query.Quantity * query.Price).sum().name("Value"))

    # If no dimensions, just apply aggregates directly
    if not dims:
        return query.aggregate(agg_exprs)

    # build expressions for group by
    group_exprs = [getattr(query, dim) for dim in dims]
    # don't overwrite query with grouped table -
    # grouped table can't be used for other query building operations
    grouped = query.group_by(group_exprs)

    # add aggregates to grouped table - returns a query
    query = grouped.aggregate(agg_exprs)

    if verbose:
        print("Group By & Aggregates Added--------------------------------")
        print(ibis.to_sql(query))

    return query

def _add_allocation(final_query: ibis.Table,
                    total_value_subquery: ibis.Table,
                    portfolio_allocation: bool,
                    agg_by_factors: bool,
                    verbose: bool = False) -> ibis.Table:
    # get value column
    value_col = final_query.Value.name('Value')
    # build allocation expression
    # - total_value is a scalar, so we can divide directly
    allocation_expr = (value_col / total_value_subquery).name("Allocation")
    query = final_query.mutate(Allocation=allocation_expr)

    if verbose:
        print("Allocation Added--------------------------------")
        print(ibis.to_sql(query))

    return query

# method parameters
metrics = ['Quantity', 'Value', 'Allocation']
dims = None #['Ticker', 'Account', 'Factor', 'Level_0','Level_1']#, 'Level_2', 'Level_3']
filters = {'Level_0': 'Equity', 'Level_1': 'US'}
portfolio_allocation = False #True  # or False, depending on your use case
verbose = True

# do metrics need to be aggregated by factors?
agg_by_factors = False
if (dims):
    agg_by_factors = (('Allocation' in metrics) or ('Value' in metrics)) and \
                     any(d.startswith('Level_') or d == 'Factor' for d in dims)

# start method prototype
unfiltered_query = _build_base_query(tables, dims, filters, verbose)
filtered_query = _apply_filters(unfiltered_query, filters, verbose)

# Group By (Dimensions)
query = _build_group_by(filtered_query, dims, metrics, agg_by_factors, verbose)

# Allocation
if ('Allocation' in metrics):
    total_value_subquery = _build_total_value_subquery(unfiltered_query,
                                                       filtered_query,
                                                       portfolio_allocation,
                                                       agg_by_factors,
                                                       verbose)
    query = _add_allocation(query,
                            total_value_subquery,
                            portfolio_allocation,
                            agg_by_factors,
                            verbose)

if verbose:
    print("Final Query --------------------------------")
    print(ibis.to_sql(query))

df = query.execute()
df

In [ ]:
# experiemnts to understand subqueries
print(ibis.__version__)

# parameters
metrics = ['Quantity', 'Value', 'Allocation']
dims = None #['Ticker', 'Account', 'Factor', 'Level_0','Level_1']#, 'Level_2', 'Level_3']
filters = {'Level_0': 'Equity', 'Level_1': 'US'}
portfolio_allocation = False #True  # or False, depending on your use case
verbose = True

# --------------------------------------------------------------
# base query
base_query = tables["holdings"].join(tables["prices"], "Ticker")
base_query = base_query.join(tables["factor_weights"], "Ticker")
base_query = base_query.join(tables["factors"], "Factor")

if verbose:
    print("\nBase Query--------------------------------")
    print(ibis.to_sql(base_query))

# --------------------------------------------------------------
# add filters
filtered_query = base_query
if filters:
    filter_conditions = []
    for dim, values in filters.items():
        # Convert single values to list
        if isinstance(values, str):
            values = [values]
        filter_conditions.append(getattr(filtered_query, dim).isin(values))
    
    # combine filter conditions
    if filter_conditions:
        combined_filter = filter_conditions[0]
        for condition in filter_conditions[1:]:
            combined_filter = combined_filter & condition
        filtered_query = base_query.filter(combined_filter)
 
if verbose:
    print("\nFiltered Query--------------------------------")
    print(ibis.to_sql(filtered_query))

# --------------------------------------------------------------
# build total value subquery
# identify correct total_value query - unfiltered or filtered
if portfolio_allocation:
    # use UNFILTERED query to calculate total value used to calculate Allocation
    total_query = base_query
else:
    # use FILTERED query to calculate total value used to calculate Allocation
    total_query = filtered_query

# build total value expression - used to calculate Allocation
if agg_by_factors:
    total_value_expr = (total_query.Quantity * total_query.Price * total_query.Weight).sum().name("Total")
else:
    total_value_expr = (total_query.Quantity * total_query.Price).sum().name("Total")

# total_value_subquery = total_query.aggregate(total_value_expr).Total.as_scalar()
total_value_subquery = total_value_expr.as_scalar()

if verbose:
    print("\nTotal Value Sub-Query --------------------------------")
    print(ibis.to_sql(total_value_subquery))

# --------------------------------------------------------------
# aggregations with group by

agg_exprs = []
if 'Quantity' in metrics:
    agg_exprs.append(filtered_query.Quantity.sum().name("Quantity"))

if 'Value' in metrics or 'Allocation' in metrics:
    if agg_by_factors:
        agg_exprs.append((filtered_query.Quantity * filtered_query.Price * filtered_query.Weight).sum().name("Value"))
    else:   
        agg_exprs.append((filtered_query.Quantity * filtered_query.Price).sum().name("Value"))

# If no dimensions, just apply aggregates directly
if not dims:
    grouped_query = filtered_query.aggregate(agg_exprs)
else:
    # build expressions for group by
    group_exprs = [getattr(filtered_query, dim) for dim in dims]
    # don't overwrite query with grouped table -
    # grouped table can't be used for other query building operations
    grouped_query = filtered_query.group_by(group_exprs)

    # add aggregates to grouped table - returns a query
    grouped_query = grouped_query.aggregate(agg_exprs)

if verbose:
    print("\nGroup By & Aggregates Added--------------------------------")
    print(ibis.to_sql(grouped_query))

# --------------------------------------------------------------
# add allocation

# get value column
value_col = grouped_query.Value.name('Value')
# build allocation expression
# - total_value is a scalar, so we can divide directly
allocation_expr = (value_col / total_value_subquery).name("Allocation")
final_query = grouped_query.mutate(Allocation=allocation_expr)

if verbose:
    print("\nAllocation Added--------------------------------")
    print(ibis.to_sql(final_query))

In [ ]:
from portopt.utils import write_table

# define column formats for write_table function
column_formats = {
    'Ticker': {'width': 14},
    'Level_0': {'width': 14},
    'Level_1': {'width': 14},
    'Level_2': {'width': 14},
    'Level_3': {'width': 14},
    'Level_4': {'width': 14},
    'Factor': {'width': 24},
    'Weight': {'width': 14, 'decimal': 3, 'type':'%'},
    'Account': {'width': 25, 'align': '<'},
    'Name': {'width': 30, 'align': '<'},
    'Short Name': {'width': 20, 'align': '<'},
    'Institution': {'width': 14},
    'Type': {'width': 14},
    'Category': {'width': 14},
    'Family': {'width': 14},
    'Owner': {'width': 14},
    'Quantity': {'width': 10, 'decimal': 3},
    'Original Ticker': {'width': 14},
    'Original Quantity': {'width': 10, 'decimal': 3},
    'Original Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Price': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Allocation': {'width': 16, 'decimal': 2, 'type':'%'}
}
column_formats
write_table(df, columns=column_formats)


In [ ]:
# simple case: calculate value allocated to each ticker
print("Value allocated to each ticker:")
print("--------------------------------")
query = tables["holdings"].join(tables["prices"], "Ticker").select("Ticker", "Account","Quantity", "Price")
print(ibis.to_sql(query))
print("--------------------------------")
query = query.mutate(Value=query.Quantity * query.Price)
print(ibis.to_sql(query))
print("--------------------------------")


In [ ]:
print("Value allocated to each factor:")
print("--------------------------------")
query = tables["holdings"].join(tables["prices"], "Ticker").join(tables["factor_weights"], "Ticker")
value_expr = (tables["holdings"].Quantity * tables["prices"].Price * tables["factor_weights"].Weight).name("Value")
query = query.select("Ticker",
                     "Account",
                     "Quantity",
                     "Price", 
                     "Weight",
                     value_expr)
print(ibis.to_sql(query))
print("--------------------------------")





In [ ]:
holdings_table = con.table("holdings")

ibis.options.interactive = True

print(holdings_table)

holdings_mutated = holdings_table.mutate(Quantity_Diff=holdings_table.Quantity - holdings_table["Original Quantity"])

print(holdings_mutated)


In [ ]:
holdings_table = con.table("holdings")
prices_table = con.table("prices")

print(holdings_table)
print(prices_table)

# join holdings and prices
holdings_prices = holdings_table.join(prices_table, "Ticker")
print(holdings_prices)

# add value columns
holdings_prices_mutated = holdings_prices.mutate(
    Value=holdings_prices.Quantity * holdings_prices.Price
)
ibis.to_sql(holdings_prices_mutated.select("Ticker", "Price", "Quantity", "Value"))

from ibis import _
ibis.to_sql(holdings_prices.filter(_.Ticker == "VOO"))
